## Dataloader

**정의된 dataset을 가지고 batch 단위로 학습하기 위해 data를 sampling 해주는 iterator**  
mini batch를 만들어주는 역할

 dataset의 전체 데이터가 batch size로 slice된다. 앞서 만들었던 dataset을 input으로 넣어주면 여러 옵션(데이터 묶기, 섞기, 알아서 병렬처리)을 통해 batch를 만들어준다. 서버에서 돌릴 때는 num_worker를 조절해서 load속도를 올릴 수 있지만, PC에서는 default로 설정해야 오류가 안난다.

dataloader에서 하는 일은 다음과 같다

- 정의된 dataset에서 batch size 만큼 data `sampling`
- sampling 된 data를 concat 해서 return

### DataLoader overview

**정의된 dataset을 가지고 batch 단위로 data를 만들어서 return 해줌**

1. dataloader에 정의된 sampler에서 batch 개수 만큼 `data sampling`을 진행
2. dataset에 정의된 getitem()을 사용해 `training data 읽어 들임`
3. dataloader에 정의된 collate_fn 내부에서 batch 개수 만큼 뽑은 data를 `concat`
    
    → 만약, concat 과정에서 data shape이 다르면 concat operation error가 남
    
    → 이 경우에는 custom collate_fn()을 만들어서 처리

In [ ]:
import torch.utils.data

tr_dataset = Dataset(train_x, train_y)
train_loader = data.DataLoader(dataset=tr_dataset, batch_size=128, num_workers=8, shuffle=True)

배치샘플러가 배치가 3이면 미니배치 사이즈 만큼의 데이터 개수 만큼 묶음. 

데이터 셋이 variable length이면 collate_fn을 이용해서
배치로 묶어야할 미니배치의 데이터를 묶어줌

In [ ]:
import torch
import torch.utils.data as data

class CustomDataLoader(data.DataLoader):
    def __init__(self, *args, **kwargs):
        super(CustomDataLoader, self).__init__(*args, **kwargs)
        self.collate_fn = _collate_fn


def _collate_fn(batch):
    
    """
    Args:
        batch: list, len(batch) = 1. See AudioDataset.__getitem__()
        DataLoader 에 배치 크기(batch size)가 있는 배치(batch) 데이터
    Returns:
        mix_torch: B x ch x T, torch.Tensor
        liens_torch : B, torch.Tentor
        src_torch: B x C x T, torch.Tensor
        
    ex)
    torch.Size([3, 6, 64000])
    tensor([64000, 64000, 64000], dtype=torch.int32)
    torch.Size([3, 2, 64000])
    """
    x_tensor=[]
    y_tensor=[]
    for i in batch[0][0]:
    	x = lbrosa.load("~.wav", sr)
        x = torch.from_numpy(pad_sequence(x))
        y = torch.from_numpy(np.load("~.npy"))
        x_tensor.append(x)
        y_tensor.append(y)
        
 
    return x_tensor, y_tensor

### **학습 시 image data를 사용할 때 주의할 사항**

- RGB image 값을 그대로 사용하지 않고 `normalize` 진행 후 사용
- normalize를 하기 위한 mean, variance 값은 학습 하는 task에 따라 다르게 사용할 수 있다
- ImageNet pretraining weight를 사용하는 경우, ImageNet에서 사용하는 mean, variance 사용

In [ ]:
def main():
	# * build DATALODER * #
    # build_train_loader(), build_val_loader()에서 dataloader를 생성 후 return
	train_loader = build_train_loader(cfg)
	val_loader = build_val_loader(cfg)

	for epoch in range(cfg.TRAIN.START_EPOCH, cfg.TRAIN.END_EPOCH+1):
        train_loss, train_metrics  = train(model, train_loader, optimizer, epoch, cfg)
        val_loss, val_metrics = validation(model, val_loader, epoch, cfg)

def train():
    # train()에서 iteration을 하며 batch 단위로 data를 받음
    # enumerate: python 반복문 사용시 해당하는 data와 index 번호를 tuple 형태로 반환

	for i_iter, inputs in enumerate(train_loader):
		# training code...

# Dataloader Parameters

Params | type | Description
---|---|---
dataset | Dataset | training data를 Loading할 dataset class 
batch_size | int | 하나의 batch당 몇개의 sample load할건지 (default = 1) 
shuffle | bool | True로 설정 시, 매 epoch마다 전체 데이터셋을 shuffle (default = False) 
sampler | Sampler | dataset으로부터 샘플링하기 위한 방법. sampler가 정의 되면 shuffle은 False
batch_sampler | Sampler | sampler와 비슷하지만, 한번에 하나의 배치에 해당하는 index sampling
num_workers | int | Data loading을 위해 사용할 subprocess의 개수
collate_fn | callable | sampling된 데이터 리스트를 하나의 mini-batch tensor로 만들어주는 함수
pin_memory | bool | True인 경우 데이터 로더는 이 데이터를 return 하기 전에 CUDA pinned memory에 copy
drop_last | bool | True로 설정 시, 전체 dataset 길이를 batch size로 나눴을 때 나눠 떨어지지 않는 데이터 셋들을 학습시킬때 제외 (default = False)
timeout | numeric | positive인 경우, worker들로부터 하나의 batch를 모으는데 정해놓은 timeout 값(default = 0)
worker_init_fn | callable | None이 아닌 경우 각 worker subprocess에서 worker id와 함께 호출 (default = None)


# Dataloader in Distributed Training

Distributed training을 할 때, sampler를 `DistributedSampler`로 사용해줘야 한다.

DistributedSampler는 전체 dataset을 gpu 개수로 나눠서 각각에 대한 sampler를 만들고 각 process에 할당한다. 

sampler는 iter method에서 dataset에 대한 index를 가지고 있는데, 하나의 sampler가 가지고 있는 index는 각 gpu에서 data가 exclusive 하게 설정된다.